In [1]:
import json

from typing import Set, Type, Union
from dataclass_maker import DataclassMaker

In [2]:
with open('schemata.json') as json_file:
    schemata = json.load(json_file)

In [3]:
specs = [sch for sch in schemata['definitions'].values() if sch['title'].endswith('TaskSpec')]
definitions = schemata['definitions']


In [4]:
dm = DataclassMaker(definitions)

In [5]:
pfile = dm.make_dataclasses_file(specs, definitions)
with open('test.py', 'w') as f:
    f.write(pfile)